<a href="https://colab.research.google.com/github/ucguate/climahub-etl/blob/NA/climahub_insercion_datos_metar_NA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Psycopg2 module installation
!pip install psycopg2
!pip install requests

In [2]:
# importing psycopg2 
import psycopg2 
from psycopg2 import Error
from pprint import pprint
# import requests for rest requests 
import requests
# importing json module
import json
# importing pandas to manage Dictionaries as pandas DataFrames
import pandas as pd

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
# creating querying functions
def show_tables(cursor):
    query = """
      SELECT *
      FROM pg_catalog.pg_tables
      WHERE schemaname != 'pg_catalog' AND 
          schemaname != 'information_schema';
    """
    cursor.execute(query)
    pprint(cursor.fetchall())

def get_table_cols(table_name):
    query = """
    SELECT column_name
      FROM information_schema.columns
        WHERE table_schema = 'public'
          AND table_name   = '{}'
     ;
    """.format(table_name)

    cursor.execute(query)
    data = cursor.fetchall()
    pprint(data)

def conn_close():
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


In [4]:
# manipulate data functions
def get_metar(zoom = 18, filter = 'prior', density = 0, taf = 'false', bbox = '-104.94370625,1.7950665583439,-60.998393750001,28.156120443236'):
  queryParams = {
      'zoom': zoom,
      'filter': filter,
      'density': density,
      'taf': taf,
      'bbox': bbox
  }
  response = requests.get("https://927d1d30.us-south.apigw.appdomain.cloud/metar-json", params=queryParams)
  #print(response.json)
  data = json.loads(response.text)
  header = data["features"][0]
  pprint(header)

  #getting column names
  variables = data["features"][1].keys()
  print(variables)

  df = pd.DataFrame(data["features"][1:], columns = variables)
  return df


# DATABASE CONNECTION and Queries Execution

In [5]:
connection = None
try:
    # Connect to an existing database
    connection = psycopg2.connect(user="climahub",
                                  password="climahub$2021",
                                  host="138.197.98.178",
                                  port="5432",
                                  database="climahub")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("🟢 CONNECTION SUCCESFUL: PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

    show_tables(cursor)
    get_table_cols('station')

except (Exception, Error) as error:
    print("❌ Error while connecting to PostgreSQL", error)


🟢 CONNECTION SUCCESFUL: PostgreSQL server information
{'user': 'climahub', 'dbname': 'climahub', 'host': '138.197.98.178', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 13.2 (Debian 13.2-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 

[('public', 'alert_types', 'climahub', None, False, False, False, False),
 ('public', 'user_groups', 'climahub', None, False, False, False, False),
 ('public', 'users', 'climahub', None, True, False, False, False),
 ('public', 'source', 'climahub', None, True, False, True, False),
 ('public', 'station', 'climahub', None, True, False, True, False),
 ('public', 'freq', 'climahub', None, True, False, True, False),
 ('public', 'variable', 'climahub', None, True, False, True, False),
 ('public', 'data_all', 'climahub', None, True, False, True, False),
 ('public', 'data_last', 'climahub'

In [7]:
get_table_cols('data_all')

[('id',),
 ('variable_id',),
 ('created_at',),
 ('data_value',),
 ('meta',),
 ('station_id',),
 ('data_date',)]


In [8]:
df=get_metar()
df

{'properties': {'bbox': '-104.9437,1.7951,-60.9984,28.1561',
                'beg_time': '1625065733',
                'data': 'METAR',
                'end_time': '1625072933',
                'wrap': ''},
 'type': 'Feature'}
dict_keys(['type', 'id', 'properties', 'geometry'])


,type,id,properties,geometry
0,Feature,470754571,"{'data': 'METAR', 'id': 'KMIA', 'site': 'Miami...","{'type': 'Point', 'coordinates': [-80.317, 25...."
1,Feature,470763077,"{'data': 'METAR', 'id': 'MMMD', 'site': 'Merid...","{'type': 'Point', 'coordinates': [-89.659, 20...."
2,Feature,470765915,"{'data': 'METAR', 'id': 'MMMX', 'site': 'Mexic...","{'type': 'Point', 'coordinates': [-99.072, 19...."
3,Feature,470766411,"{'data': 'METAR', 'id': 'MROC', 'site': 'San J...","{'type': 'Point', 'coordinates': [-84.218, 9.9..."
4,Feature,470750715,"{'data': 'METAR', 'id': 'MWCR', 'site': 'Grand...","{'type': 'Point', 'coordinates': [-81.357, 19...."
...,...,...,...,...
226,Feature,470760827,"{'data': 'METAR', 'id': 'KCLW', 'site': 'Clear...","{'type': 'Point', 'coordinates': [-82.756, 27...."
227,Feature,470764755,"{'data': 'METAR', 'id': 'KNQX', 'site': 'Key W...","{'type': 'Point', 'coordinates': [-81.683, 24...."
228,Feature,470768561,"{'data': 'METAR', 'id': 'KPMP', 'site': 'Pompa...","{'type': 'Point', 'coordinates': [-80.111, 26...."
229,Feature,470764945,"{'data': 'METAR', 'id': 'KRAS', 'site': 'Port ...","{'type': 'Point', 'coordinates': [-97.085, 27...."


In [11]:
df['properties'].iloc[14]

{'altim': 1020.4,
 'ceil': 20,
 'cldBas1': '20',
 'cldBas2': '80',
 'cldCvg1': 'BKN',
 'cldCvg2': 'OVC',
 'cover': 'OVC',
 'data': 'METAR',
 'dewp': 17.0,
 'fltcat': 'MVFR',
 'id': 'MMTC',
 'obsTime': '2021-06-30T16:42:00Z',
 'prior': 1,
 'rawOb': 'MMTC 301642Z 00000KT 6SM BKN020 OVC080 26/17 A3013 RMK 8/11/ HZY',
 'site': 'Torreon/Tinoco Intl',
 'temp': 26.0,
 'visib': 6.0,
 'wdir': 0,
 'wspd': 0}

In [12]:
src=1
var=dict()
var['tmp'] =3       ##TMP	Temperatura	1, Horaria	2021-05-07				
var['dewp']=4	      ##DWP	Punto de Rocío	1, Horaria	2021-05-07				
                     ##5	PCP	Precipitación	1, Horaria	2021-05-07				
var['wdir']=7	      ##WND	Dirección del Viento	1, Horaria	2021-05-07				
var['wspd']=8	      ##WNS	Velocidad del viento KPH	1, Horaria	2021-05-07				
var['wgst']=9	      ##WNG	Ráfaga de viento KPH	1, Horaria	2021-05-07				
#var['slp']=10	     ##PRS	Presión Atmosférica	1, Horaria	2021-05-07				
var['rawOb']=11         ##	RAW	Objeto de datos o mensaje. (Si está disponible)	1, Horaria	2021-05-07				
var['wx']=12	       ##WX	Tiempo Presente (METAR)	1, Horaria	2021-05-07

variables_metar=['tmp','dewp','wdir','wspd','wgst','rawOb','wx']

In [ ]:

for index, row in df.iterrows():
    #print(index,row['properties']['id'])
    ##se creo un indice único de source_id,source_station_id
   # print('Processing: '+row['properties']['id']+' '+row['properties']['site'])
    query='insert into station (source_id,station_name,latitude,longitude,source_station_id,original_id) values ('
    query=query+"{},'{}',{},{},'{}','{}') on conflict do nothing;".format(src,row['properties']['site'],row['geometry']['coordinates'][1],row['geometry']['coordinates'][0],row['properties']['id'],row['properties']['id'])
   # print(query)
    cursor.execute(query)
    query_data=''
    for variable in variables_metar:
      if  variable in row['properties']:
          query_data=query_data+"INSERT INTO data_all(variable_id,data_date,data_value, station_id) VALUES({},'{}','{}',{}) on conflict do nothing;".format(var[variable],row['properties']['obsTime'],row['properties'][variable],'(SELECT id FROM station WHERE source_station_id = \''+row['properties']['id']+'\')')
   # print(query_data)
    cursor.execute(query_data)
    connection.commit()
    
        

In [ ]:
conn_close()

PostgreSQL connection is closed
